In [1]:
%load_ext lab_black

In [2]:
import numpy as np
import pandas as pd
import itertools as it
import ipyparallel as ipp

from random import normalvariate

import gc
import timeit
import sys
import os

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Revoking stored reported data
# https://ipython.org/ipython-doc/rel-0.12/config/extensions/storemagic.html

%store -r data_Averoy
%store -r data_Tingvoll
%store -r data_Surnadal
%store -r data_Sunndal
%store -r data_Vestnes
%store -r data_Laerdal

# Constructing hypothetical (synthetic) populations

In [4]:
# Create subdirectory /synthetic_data if it does not exist
path = "./synthetic_data"

# Check whether directory already exists
if not os.path.exists(path):
    os.mkdir(path)
    print("Folder %s created!" % path)
else:
    print("Folder %s already exists" % path)

Folder ./synthetic_data already exists


In [5]:
# Setting up the cluster
cluster = ipp.Cluster(n=52)
cluster.start_cluster_sync()
rc = cluster.connect_client_sync()
rc.wait_for_engines(52)
rc.ids
dv = rc[:]

Using existing profile dir: '/Users/stigomholt/.ipython/profile_default'
Starting 52 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


<Cluster(cluster_id='1739956107-4x1p', profile='default', controller=<running>, engine_sets=['1739956108'])>

  0%|          | 0/52 [00:00<?, ?engine/s]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51]

In [6]:
with dv.sync_imports():
    from random import normalvariate
    import numpy as np

importing normalvariate from random on engine(s)
importing numpy on engine(s)


In [7]:
a = 10
b = 5
dv.push({"a": a, "b": b})

<AsyncResult(_push): pending>

In [8]:
# To ensure the variables are on engine 0
rc[0]["a"]

10

In [9]:
rc[50]["b"]

5

### Calculate the initial sizes of all six population categories

In [10]:
%%px
def calculate_initial_size_of_each_category(population_data):
    total_number_calves = population_data[0] * population_data[3]
    total_number_adults = population_data[0] * (1 - population_data[3])
    total_number_stags = total_number_adults / (population_data[1] + 1)
    total_number_hinds = total_number_adults - total_number_stags

    female_calves_survivors = total_number_calves / 2
    male_calves_survivors = total_number_calves / 2

    young_hinds_survivors = total_number_hinds * population_data[4]
    adult_hinds_survivors = total_number_hinds - young_hinds_survivors

    young_stags_survivors = total_number_stags * population_data[5]
    adult_stags_survivors = total_number_stags - young_stags_survivors
    return (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )

In [11]:
%%px
def winter_mortality_all_categories(
    female_calves, young_hinds, adult_hinds, male_calves, young_stags, adult_stags
):
    # Assumed winter survival parameters for all six population categories
    w_survival = [
        normalvariate(0.8, 0.04),
        normalvariate(0.92, 0.03),
        normalvariate(0.93, 0.03),
        normalvariate(0.8, 0.04),
        normalvariate(0.87, 0.04),
        normalvariate(0.93, 0.03),
    ]
    if w_survival[0] >= 1.0:
        fraction_female_calves_surviving_winter = 1.0
    else:
        fraction_female_calves_surviving_winter = w_survival[0]

    if w_survival[1] >= 1.0:
        fraction_young_hinds_surviving_winter = 1.0
    else:
        fraction_young_hinds_surviving_winter = w_survival[1]

    if w_survival[2] >= 1.0:
        fraction_adult_hinds_surviving_winter = 1.0
    else:
        fraction_adult_hinds_surviving_winter = w_survival[2]

    if w_survival[3] >= 1.0:
        fraction_male_calves_surviving_winter = 1.0
    else:
        fraction_male_calves_surviving_winter = w_survival[3]

    if w_survival[4] >= 1.0:
        fraction_young_stags_surviving_winter = 1.0
    else:
        fraction_young_stags_surviving_winter = w_survival[4]

    if w_survival[5] >= 1.0:
        fraction_adult_stags_surviving_winter = 1.0
    else:
        fraction_adult_stags_surviving_winter = w_survival[5]

    female_calves_survivors = female_calves * fraction_female_calves_surviving_winter
    young_hinds_survivors = young_hinds * fraction_young_hinds_surviving_winter
    adult_hinds_survivors = adult_hinds * fraction_adult_hinds_surviving_winter

    male_calves_survivors = male_calves * fraction_male_calves_surviving_winter
    young_stags_survivors = young_stags * fraction_young_stags_surviving_winter
    adult_stags_survivors = adult_stags * fraction_adult_stags_surviving_winter

    return (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        fraction_female_calves_surviving_winter,
        fraction_young_hinds_surviving_winter,
        fraction_adult_hinds_surviving_winter,
        fraction_male_calves_surviving_winter,
        fraction_young_stags_surviving_winter,
        fraction_adult_stags_surviving_winter,
    )

### Spring update of population categories 

In [12]:
%%px
def spring_update_of_categories(
    female_calves_survivors,
    young_hinds_survivors,
    adult_hinds_survivors,
    male_calves_survivors,
    young_stags_survivors,
    adult_stags_survivors,
):
    # Surviving female calves become one year old young hinds
    young_hinds = female_calves_survivors

    # Surviving young hinds born almost two years ago become adult hinds
    adult_hinds = young_hinds_survivors + adult_hinds_survivors

    # Surviving male calves become one year old young stags
    young_stags = male_calves_survivors

    # Surviving young stags born almost two years ago become adult stags
    adult_stags = young_stags_survivors + adult_stags_survivors
    return young_hinds, adult_hinds, young_stags, adult_stags

### Calculate number of female and male calves born in early summer

In [13]:
%%px
def early_summer_breeding(young_hinds_survivors, adult_hinds_survivors):

    # Assumed fertility parameters for young and adult hinds
    calves_per_young_hind = normalvariate(0.4, 0.05)

    calves_per_adult_hind = normalvariate(0.975, 0.008)
    if calves_per_adult_hind >= 1.0:
        calves_per_adult_hind = 1.0

    female_calves_born = (
        young_hinds_survivors * calves_per_young_hind / 2
        + adult_hinds_survivors * calves_per_adult_hind / 2
    )
    male_calves_born = female_calves_born
    return (
        female_calves_born,
        male_calves_born,
        calves_per_young_hind,
        calves_per_adult_hind,
    )

### Calculate putative emigration of stags

In [14]:
%%px
def emigration_of_stags(stags_per_hind_threshold, adult_stags, adult_hinds):
    """
    Does not account for the hypothesis that migration is primarily sported by
    the two years old stags. The way we deal with the emigration issue is highly
    idealized as we say that when the number of adult stags per adult hind is
    larger than a certain number, then stags will emigrate until the ratio falls
    below this number. The underlying hypothesis is that the stags are
    capable of making an assessment of their potential breeding success based on
    perceptual data during the winter and the spring and act on this information
    by use of a built in algorithm.
    """

    if stags_per_hind_threshold == 0 or adult_hinds < 1 or adult_stags < 1:
        number_of_migrants = 0
    else:
        number_of_migrants = adult_stags - stags_per_hind_threshold * adult_hinds
        if number_of_migrants <= 0:
            number_of_migrants = 0

    if number_of_migrants > 0:
        migration_percentage = 100 * number_of_migrants / adult_stags
    else:
        migration_percentage = 0

    adult_stags = adult_stags - number_of_migrants

    return adult_stags, number_of_migrants, migration_percentage

### Calculate number of individuals in each category surviving the hunting season

In [15]:
%%px
def transform_harvested_percentages_to_numbers(
    total_harvest,
    fraction_female_calves_harvested,
    fraction_male_calves_harvested,
    fraction_young_hinds_harvested,
    fraction_adult_hinds_harvested,
    fraction_young_stags_harvested,
    fraction_adult_stags_harvested,
):
    number_harvested_female_calves = np.round(
        np.multiply(np.array(fraction_female_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_hinds = np.round(
        np.multiply(np.array(fraction_young_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_hinds = np.round(
        np.multiply(np.array(fraction_adult_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_male_calves = np.round(
        np.multiply(np.array(fraction_male_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_stags = np.round(
        np.multiply(np.array(fraction_young_stags_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_stags = np.round(
        np.multiply(np.array(fraction_adult_stags_harvested), np.array(total_harvest))
        / 100
    )
    return (
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
    )

In [16]:
%%px
def survivors_after_hunting(
    number_harvested_female_calves,
    number_harvested_young_hinds,
    number_harvested_adult_hinds,
    number_harvested_male_calves,
    number_harvested_young_stags,
    number_harvested_adult_stags,
    female_calves_born,
    male_calves_born,
    young_hinds,
    adult_hinds,
    young_stags,
    adult_stags,
):
    female_calves_survivors = female_calves_born - number_harvested_female_calves
    young_hinds_survivors = young_hinds - number_harvested_young_hinds
    adult_hinds_survivors = adult_hinds - number_harvested_adult_hinds

    male_calves_survivors = male_calves_born - number_harvested_male_calves
    young_stags_survivors = young_stags - number_harvested_young_stags
    adult_stags_survivors = adult_stags - number_harvested_adult_stags

    if female_calves_survivors < 0:
        female_calves_survivors = 0
    if young_hinds_survivors < 0:
        young_hinds_survivors = 0
    if adult_hinds_survivors < 0:
        adult_hinds_survivors = 0

    if male_calves_survivors < 0:
        male_calves_survivors = 0
    if young_stags_survivors < 0:
        young_stags_survivors = 0
    if adult_stags_survivors < 0:
        adult_stags_survivors = 0
    return (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )

### Project population development from one year to the next

In [17]:
%%px
def project_population_development(
    number_harvested_female_calves,
    number_harvested_young_hinds,
    number_harvested_adult_hinds,
    number_harvested_male_calves,
    number_harvested_young_stags,
    number_harvested_adult_stags,
    female_calves_survivors,
    young_hinds_survivors,
    adult_hinds_survivors,
    male_calves_survivors,
    young_stags_survivors,
    adult_stags_survivors,
    stags_per_hind_threshold,
    year,
):

    # Let Mother winter take her toll
    (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        fraction_female_calves_surviving_winter,
        fraction_young_hinds_surviving_winter,
        fraction_adult_hinds_surviving_winter,
        fraction_male_calves_surviving_winter,
        fraction_young_stags_surviving_winter,
        fraction_adult_stags_surviving_winter,
    ) = winter_mortality_all_categories(
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )

    # Let calves become young ones and young ones become adults
    young_hinds, adult_hinds, young_stags, adult_stags = spring_update_of_categories(
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    )
    # Total population in the spring
    total_population_in_spring = young_hinds + adult_hinds + young_stags + adult_stags

    # Time for giving birth
    (
        female_calves_born,
        male_calves_born,
        calves_per_young_hind,
        calves_per_adult_hind,
    ) = early_summer_breeding(young_hinds_survivors, adult_hinds_survivors)

    # A tough decision made by stags
    adult_stags, number_of_migrants, migration_percentage = emigration_of_stags(
        stags_per_hind_threshold, adult_stags, adult_hinds
    )

    # Calculate ratio hinds per stag just before hunting season
    if (young_stags + adult_stags) > 1.0:
        hinds_per_stag_before_hunting = (young_hinds + adult_hinds) / (
            young_stags + adult_stags
        )
    else:
        hinds_per_stag_before_hunting = "NaN"

    # Total population just before hunting season starts
    total_population_before_hunting = (
        female_calves_born
        + young_hinds
        + adult_hinds
        + male_calves_born
        + young_stags
        + adult_stags
    )

    # Oh Deer, life is becoming dangerous
    (
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
    ) = survivors_after_hunting(
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
        female_calves_born,
        male_calves_born,
        young_hinds,
        adult_hinds,
        young_stags,
        adult_stags,
    )

    # Total population in December after end of hunting season
    total_population_after_hunting = (
        female_calves_survivors
        + young_hinds_survivors
        + adult_hinds_survivors
        + male_calves_survivors
        + young_stags_survivors
        + adult_stags_survivors
    )

    # Calculate ratio hinds per stag in December after end of hunting season
    if (young_stags_survivors + adult_stags_survivors) > 1:
        hinds_per_stag_after_hunting = (
            young_hinds_survivors + adult_hinds_survivors
        ) / (young_stags_survivors + adult_stags_survivors)
    else:
        hinds_per_stag_after_hunting = "NaN"

    return (
        total_population_in_spring,
        number_of_migrants,
        migration_percentage,
        hinds_per_stag_before_hunting,
        total_population_before_hunting,
        female_calves_survivors,
        young_hinds_survivors,
        adult_hinds_survivors,
        male_calves_survivors,
        young_stags_survivors,
        adult_stags_survivors,
        total_population_after_hunting,
        hinds_per_stag_after_hunting,
        fraction_female_calves_surviving_winter,
        fraction_young_hinds_surviving_winter,
        fraction_adult_hinds_surviving_winter,
        fraction_male_calves_surviving_winter,
        fraction_young_stags_surviving_winter,
        fraction_adult_stags_surviving_winter,
        calves_per_young_hind,
        calves_per_adult_hind,
    )

### Transform harvest percentages to numbers

In [18]:
def transform_harvested_percentages_to_numbers(
    total_harvest,
    fraction_female_calves_harvested,
    fraction_male_calves_harvested,
    fraction_young_hinds_harvested,
    fraction_adult_hinds_harvested,
    fraction_young_stags_harvested,
    fraction_adult_stags_harvested,
):
    number_harvested_female_calves = np.round(
        np.multiply(np.array(fraction_female_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_hinds = np.round(
        np.multiply(np.array(fraction_young_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_hinds = np.round(
        np.multiply(np.array(fraction_adult_hinds_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_male_calves = np.round(
        np.multiply(np.array(fraction_male_calves_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_young_stags = np.round(
        np.multiply(np.array(fraction_young_stags_harvested), np.array(total_harvest))
        / 100
    )
    number_harvested_adult_stags = np.round(
        np.multiply(np.array(fraction_adult_stags_harvested), np.array(total_harvest))
        / 100
    )
    return (
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
    )

### Scripts for L2_norm_spring_count analysis

In [19]:
%%px
def L2_norm_spring_count(
    population_log,
    empirical_observations,
    number_of_years,
):
    """
    Calculating the minimum L2 norm between synthetic data and
    reported data
    """
    import numpy as np
    # Establish the two time series to be compared
    spring_count_e = np.array(empirical_observations)
    spring_count_record = [x[17] for x in population_log] #17:position spring population
    spring_count_p = np.array(spring_count_record)
    
    if spring_count_p[-1] > 10:
        # Find the scaling factor that minimises RSS and do the scaling
        scaling_factor = np.sum(np.multiply(spring_count_p, spring_count_e)) / np.sum(
            np.multiply(spring_count_p, spring_count_p)
        )
    else:
        scaling_factor = 100.
    
    spring_count_p_scaled = scaling_factor * spring_count_p

    # Find minimum Residual Sum Square value
    L2_norm_min = (np.sum((spring_count_p_scaled - spring_count_e) ** 2))**(1/2)

    return scaling_factor, L2_norm_min

### Main ipy parallel script

In [20]:
@dv.remote(block=True)
def ipy_master():
    import numpy as np

    # Initial preparations
    array_top_hits = np.zeros((number_of_top_hits, number_of_years, 34))
    new_val = np.arange(10000, 10000 + number_of_top_hits, 1)
    scaling_factor = 500
    for i in range(array_top_hits.shape[0]):
        array_top_hits[i, -1, array_top_hits.shape[2] - 2] = scaling_factor
        array_top_hits[i, -1, array_top_hits.shape[2] - 1] = new_val[i]

    # Start going through states
    for state in init_chunk:
        # Run through for replicate number z
        for z in range(number_of_replicates):
            replicate_number = z

            population_log = []

            # Calculate size of each category in the initial population December 31 year -1
            (
                female_calves_survivors,
                young_hinds_survivors,
                adult_hinds_survivors,
                male_calves_survivors,
                young_stags_survivors,
                adult_stags_survivors,
            ) = calculate_initial_size_of_each_category(state)

            # Record initial sizes of the six categories
            female_calves_initially = female_calves_survivors
            young_hinds_initially = young_hinds_survivors
            adult_hinds_initially = adult_hinds_survivors
            male_calves_initially = male_calves_survivors
            young_stags_initially = young_stags_survivors
            adult_stags_initially = adult_stags_survivors

            # Run through all years in the time series for each initial state
            for j in range(number_of_years):
                (
                    total_population_in_spring,
                    number_of_migrants,
                    migration_percentage,
                    hinds_per_stag_before_hunting,
                    total_population_before_hunting,
                    female_calves_survivors,
                    young_hinds_survivors,
                    adult_hinds_survivors,
                    male_calves_survivors,
                    young_stags_survivors,
                    adult_stags_survivors,
                    total_population_after_hunting,
                    hinds_per_stag_after_hunting,
                    fraction_female_calves_surviving_winter,
                    fraction_young_hinds_surviving_winter,
                    fraction_adult_hinds_surviving_winter,
                    fraction_male_calves_surviving_winter,
                    fraction_young_stags_surviving_winter,
                    fraction_adult_stags_surviving_winter,
                    calves_per_young_hind,
                    calves_per_adult_hind,
                ) = project_population_development(
                    number_harvested_female_calves[j],
                    number_harvested_young_hinds[j],
                    number_harvested_adult_hinds[j],
                    number_harvested_male_calves[j],
                    number_harvested_young_stags[j],
                    number_harvested_adult_stags[j],
                    female_calves_survivors,
                    young_hinds_survivors,
                    adult_hinds_survivors,
                    male_calves_survivors,
                    young_stags_survivors,
                    adult_stags_survivors,
                    state[2],
                    j,
                )

                new_record = [
                    state[0],
                    state[1],
                    state[2],
                    female_calves_initially,
                    young_hinds_initially,
                    adult_hinds_initially,
                    male_calves_initially,
                    young_stags_initially,
                    adult_stags_initially,
                    replicate_number,
                    first_year + j,
                    female_calves_survivors,
                    young_hinds_survivors,
                    adult_hinds_survivors,
                    male_calves_survivors,
                    young_stags_survivors,
                    adult_stags_survivors,
                    total_population_in_spring,
                    total_population_before_hunting,
                    total_population_after_hunting,
                    hinds_per_stag_before_hunting,
                    hinds_per_stag_after_hunting,
                    migration_percentage,
                    number_of_migrants,
                    fraction_female_calves_surviving_winter,
                    fraction_young_hinds_surviving_winter,
                    fraction_adult_hinds_surviving_winter,
                    fraction_male_calves_surviving_winter,
                    fraction_young_stags_surviving_winter,
                    fraction_adult_stags_surviving_winter,
                    calves_per_young_hind,
                    calves_per_adult_hind,
                ]

                population_log.append(new_record)

            # Do L2_norm fitting of the number_of_years population trajectory
            scaling_factor, L2_value = L2_norm_spring_count(
                population_log,
                spring_counts,
                number_of_years,
            )

            # Find max L2 norm in array_top_hits and exchange if better fit
            if L2_value < array_top_hits[np.argmax(array_top_hits[:, -1, -1]), -1, -1]:
                # Add scaling_factor and L2 value at the end of each record in population_log
                # and exchange number_of_years records
                for k in range(len(population_log)):
                    population_log[k].extend([scaling_factor, L2_value])
                    array_top_hits[
                        np.argmax(array_top_hits[:, -1, -1]), k, :
                    ] = population_log[k]

    # Make final sorting
    # array_top_hits = array_top_hits[array_top_hits[:,:,-1].argsort()]

    return array_top_hits

In [21]:
%%time
    
# Number of replicates of each initial condition 
number_of_replicates = 1 #50 #30
number_of_top_hits = 2000 # from each engine
dv.push(dict(number_of_replicates = number_of_replicates, number_of_top_hits = number_of_top_hits))

# To ensure the variables are on engines 
#print(rc[0]['number_of_replicates'],rc[50]['number_of_top_hits']) 

# Using all six municipalties
municipalities = ["Tingvoll", "Surnadal", "Sunndal", "Vestnes", "Laerdal"]
data_municipality = [
    data_Tingvoll,
    data_Surnadal,
    data_Sunndal,
    data_Vestnes,
    data_Laerdal,
]

for q in range(len(municipalities)):
    [
        municipality,
        first_year,
        last_year,
        number_of_years,
        years,
        seen_deer_obs,
        seen_deer_obs_forest,
        seen_deer_obs_farmland,
        hinds_per_stag_obs,
        total_harvest,
        total_quota,
        fraction_female_calves_harvested,
        fraction_young_hinds_harvested,
        fraction_adult_hinds_harvested,
        fraction_male_calves_harvested,
        fraction_young_stags_harvested,
        fraction_adult_stags_harvested,
        spring_counts,
    ] = data_municipality[q]
    
    dv.push(dict(municipality = municipality ,
            first_year = first_year,
            last_year = last_year,
            number_of_years = number_of_years,
            years = years,
            seen_deer_obs = seen_deer_obs,
            seen_deer_obs_forest = seen_deer_obs_forest,
            seen_deer_obs_farmland = seen_deer_obs_farmland,
            hinds_per_stag_obs = hinds_per_stag_obs,
            total_harvest = total_harvest,
            fraction_female_calves_harvested = fraction_female_calves_harvested,
            fraction_young_hinds_harvested = fraction_young_hinds_harvested,
            fraction_adult_hinds_harvested = fraction_adult_hinds_harvested,
            fraction_male_calves_harvested = fraction_male_calves_harvested,
            fraction_young_stags_harvested = fraction_young_stags_harvested,
            fraction_adult_stags_harvested = fraction_adult_stags_harvested,
            spring_counts = spring_counts))

    # To ensure the variables are on engines 
    #print(rc[5]['years'],rc[45]['total_harvest']) 
    
    # Setting up all initial conditions to be run with calibrated init_pop
    # Make all combinations of the six initial conditions
    #initial_population = np.arange(init_pop_range[q][0],init_pop_range[q][1], 10)
    initial_population = np.arange(900, 4000, 10)
    #initial_hinds_per_stag = np.array([1.7]) # uses the inverse of the estimated stags_per_hind threshold
    initial_hinds_per_stag = np.arange(1.3, 2.2, 0.1)
    init_frac_calves = np.arange(0.2, 0.355, 0.01)
    init_frac_y_hinds = np.arange(0.09, 0.275, 0.01)  # of total number hinds
    init_frac_y_stags = np.arange(0.18, 0.545, 0.02)  # of total number stags
    #stags_per_hind_threshold = np.array([0.59])  # not really initial condition, but initial premise
    stags_per_hind_threshold = np.arange(0.5, 0.8, 0.05)

    all_initial_states = np.array(
        np.meshgrid(
            initial_population,
            initial_hinds_per_stag,
            stags_per_hind_threshold,
            init_frac_calves,
            init_frac_y_hinds,
            init_frac_y_stags,
        )
    ).T.reshape(-1, 6)

    dv.scatter("init_chunk", all_initial_states)
    # To ensure the variables are on engines 
    #check = rc[0]['init_chunk']
    #print("len(init_chunk):",len(check)) 
    
    # Transform reported hunting percentages for all years to numbers
    (
        number_harvested_female_calves,
        number_harvested_young_hinds,
        number_harvested_adult_hinds,
        number_harvested_male_calves,
        number_harvested_young_stags,
        number_harvested_adult_stags,
    ) = transform_harvested_percentages_to_numbers(
        total_harvest,
        fraction_female_calves_harvested,
        fraction_male_calves_harvested,
        fraction_young_hinds_harvested,
        fraction_adult_hinds_harvested,
        fraction_young_stags_harvested,
        fraction_adult_stags_harvested,
    )

    dv.push(dict(number_harvested_female_calves = number_harvested_female_calves,
        number_harvested_young_hinds = number_harvested_young_hinds,
        number_harvested_adult_hinds= number_harvested_adult_hinds,
        number_harvested_male_calves = number_harvested_male_calves,
        number_harvested_young_stags= number_harvested_young_stags,
        number_harvested_adult_stags = number_harvested_adult_stags))
    
    # To ensure the variables are on engines 
    #print(rc[10]['number_harvested_young_hinds'],rc[45]['number_harvested_adult_stags']) 
    
    # Getting array_top_hits from all engines in a 3d array that needs to be flattened
    array_top_hits = np.array(ipy_master())
    aggregated_arr = array_top_hits.reshape(-1, array_top_hits.shape[-1])
          

    # Make pandas dataframe for given municipality
    column_values = [
        "init_pop",
        "init_hps",
        "sh_threshold",
        "i_fcalves",
        "i_yhinds",
        "i_ahinds",
        "i_mcalves",
        "i_ystags",
        "i_astags",
        "rep_num",
        "obs_year",
        "f_calves",
        "y_hinds",
        "a_hinds",
        "m_calves",
        "y_stags",
        "a_stags",
        "spring_pop",
        "tot_pop_bh",
        "tot_pop_ah",
        "hps_bh",
        "hps_ah",
        "mig%",
        "num_migs",
        "ws_fc",
        "ws_yh",
        "ws_ah",
        "ws_mc",
        "ws_ys",
        "ws_as",
        "c_yh",
        "c_ah",
        "scaling",
        "L2_norm",
    ]
    pop_data = pd.DataFrame(data=aggregated_arr, columns=column_values)
    pop_data = pop_data.astype(
        {
            "init_pop": "int",
            "rep_num": "int",
            "obs_year": "int",
            "i_fcalves": "int",
            "i_yhinds": "int",
            "i_ahinds": "int",
            "i_mcalves": "int",
            "i_ystags": "int",
            "i_astags": "int",
            "f_calves": "int",
            "y_hinds": "int",
            "a_hinds": "int",
            "m_calves": "int",
            "y_stags": "int",
            "a_stags": "int",
            "spring_pop": "int",
            "tot_pop_bh": "int",
            "tot_pop_ah": "int",
            "num_migs": "int",
        }
    ).round(14)

    # Sort and trim the data frame
    #pop_data_sorted = pop_data.sort_values(by=["L2_norm"]).reset_index(drop=True)
    pop_data_sorted = pop_data.sort_values(['L2_norm','obs_year'],ascending=[True, True]).reset_index(drop=True)
        
    # Storing all top hits to allow variation in the number of top hits later on
    save_dir = "synthetic_data/"
    pop_data_sorted.to_pickle(
        save_dir + "top_hits_" + municipality + "_spring_count_stoch_par" + ".pkl"
    )


    print("Municipality = ", municipality)
    pd.set_option('display.max_columns', None)
    display(pop_data_sorted)
    print()
    print()

Municipality =  Tingvoll


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,rep_num,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,scaling,L2_norm
0,2240,2.1,0.75,235,179,1018,235,285,285,0,2007,407,88,964,385,135,446,1946,2884,2427,1.725641,1.810821,0.000000,0,0.753384,0.859614,0.884015,0.837917,0.882588,0.929368,0.421567,0.968260,0.349640,515.766048
1,2240,2.1,0.75,235,179,1018,235,285,285,0,2008,395,238,872,371,209,432,2107,3035,2521,1.738707,1.729085,0.000000,0,0.828451,0.903238,0.954938,0.712990,0.765718,0.876995,0.403521,0.971927,0.349640,515.766048
2,2240,2.1,0.75,235,179,1018,235,285,285,0,2009,331,208,851,327,220,525,2174,3020,2466,1.473686,1.421412,0.000000,0,0.757600,0.950247,0.881224,0.743958,0.914742,0.948744,0.394821,0.983466,0.349640,515.766048
3,2240,2.1,0.75,235,179,1018,235,285,285,0,2010,339,172,875,339,197,602,2220,3104,2528,1.353473,1.309192,0.000000,0,0.765154,0.966837,0.964459,0.825596,0.858104,0.920425,0.387641,0.979917,0.349640,515.766048
4,2240,2.1,0.75,235,179,1018,235,285,285,0,2011,354,211,843,344,205,653,2289,3132,2613,1.272464,1.228182,1.664783,12,0.869175,0.934432,0.934835,0.786578,0.886684,0.948694,0.384784,0.969535,0.349640,515.766048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767995,2170,1.9,0.70,271,234,831,271,179,381,0,2019,457,262,1131,449,276,800,2935,4016,3377,1.325584,1.294038,4.002087,37,0.856198,0.913492,0.958243,0.816019,0.848236,0.919723,0.367912,0.981133,0.362736,620.887571
1767996,2170,1.9,0.70,271,234,831,271,179,381,0,2020,447,235,1081,420,268,782,2912,3921,3236,1.299448,1.253198,7.444971,70,0.782069,0.888894,0.897255,0.805836,0.928355,0.858687,0.355171,0.981952,0.362736,620.887571
1767997,2170,1.9,0.70,271,234,831,271,179,381,0,2021,394,253,1011,381,257,730,2887,3723,3028,1.315606,1.281253,17.624565,174,0.838586,0.914976,0.878858,0.844834,0.926425,0.948408,0.457199,0.959183,0.362736,620.887571
1767998,2170,1.9,0.70,271,234,831,271,179,381,0,2022,384,212,993,401,226,720,2645,3529,2939,1.333447,1.273292,11.044340,99,0.798241,0.919893,0.896714,0.768529,0.882534,0.918032,0.451222,0.966966,0.362736,620.887571




Municipality =  Surnadal


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,rep_num,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,scaling,L2_norm
0,1730,1.7,0.75,233,190,604,233,102,364,0,2007,254,120,650,242,124,334,1496,2095,1726,1.499337,1.680135,0.000000,0,0.736549,0.916073,0.911112,0.781004,0.894875,0.888268,0.375215,0.969642,0.479052,273.938025
1,1730,1.7,0.75,233,190,604,233,102,364,0,2008,247,154,612,222,120,335,1521,2118,1692,1.448225,1.683941,0.000000,0,0.808340,0.937707,0.894540,0.796960,0.914805,0.940850,0.250602,0.977417,0.479052,273.938025
2,1730,1.7,0.75,233,190,604,233,102,364,0,2009,259,133,634,258,98,316,1507,2153,1700,1.645953,1.853657,0.000000,0,0.814789,0.883539,0.978336,0.726529,0.795856,0.931274,0.431741,0.980337,0.479052,273.938025
3,1730,1.7,0.75,233,190,604,233,102,364,0,2010,246,141,628,249,137,271,1522,2170,1673,1.649034,1.883332,0.000000,0,0.834912,0.877359,0.966832,0.775942,0.839190,0.922420,0.386716,0.981919,0.479052,273.938025
4,1730,1.7,0.75,233,190,604,233,102,364,0,2011,235,135,586,232,149,302,1468,2074,1641,1.505461,1.599065,0.000000,0,0.786497,0.871012,0.900783,0.823665,0.859489,0.969589,0.448147,0.972544,0.479052,273.938025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767995,1600,1.8,0.60,200,115,655,200,111,317,0,2019,237,172,647,237,147,372,1754,2415,1814,1.446739,1.575062,2.359317,11,0.786842,0.926508,0.888936,0.773546,0.883320,0.959991,0.314510,0.975296,0.485738,357.357415
1767996,1600,1.8,0.60,200,115,655,200,111,317,0,2020,247,127,635,248,113,343,1614,2221,1714,1.458587,1.672891,6.707637,32,0.750400,0.896408,0.935792,0.788924,0.915195,0.949239,0.357768,0.965527,0.485738,357.357415
1767997,1600,1.8,0.60,200,115,655,200,111,317,0,2021,238,152,599,243,112,325,1532,2160,1670,1.466438,1.719773,0.000000,0,0.805059,0.922930,0.935722,0.787495,0.876893,0.952931,0.429216,0.971665,0.485738,357.357415
1767998,1600,1.8,0.60,200,115,655,200,111,317,0,2022,250,147,624,240,157,299,1494,2090,1719,1.448813,1.688633,0.000000,0,0.800258,0.901582,0.927228,0.873965,0.884716,0.917964,0.395710,0.974778,0.485738,357.357415




Municipality =  Sunndal


...



Municipality =  Vestnes


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,rep_num,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,scaling,L2_norm
0,1860,1.6,0.80,186,91,824,186,148,423,0,2007,316,95,791,311,88,460,1684,2463,2064,1.484979,1.617199,0.000000,0,0.827752,0.980659,0.925538,0.803938,0.896883,0.932238,0.450227,0.968047,0.374836,346.519526
1,1860,1.6,0.80,186,91,824,186,148,423,0,2008,291,200,739,308,167,418,1844,2607,2126,1.499644,1.605362,0.000000,0,0.865120,0.855091,0.948620,0.764957,0.816607,0.928695,0.473004,0.964755,0.374836,346.519526
2,1860,1.6,0.80,186,91,824,186,148,423,0,2009,281,160,726,278,181,403,1845,2568,2032,1.439827,1.515326,0.000000,0,0.805654,0.901468,0.910126,0.805593,0.798966,0.893998,0.349581,0.979548,0.374836,346.519526
3,1860,1.6,0.80,186,91,824,186,148,423,0,2010,255,166,701,253,176,455,1871,2587,2007,1.346671,1.374451,0.000000,0,0.867722,0.945960,0.932972,0.907059,0.920529,0.935473,0.345589,0.978981,0.374836,346.519526
4,1860,1.6,0.80,186,91,824,186,148,423,0,2011,282,151,729,278,144,509,1825,2506,2095,1.270626,1.346436,0.000000,0,0.835545,0.918518,0.934591,0.804096,0.897641,0.971733,0.342650,0.957474,0.374836,346.519526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767995,2070,1.8,0.75,351,228,649,351,117,370,0,2019,327,169,830,293,146,621,2218,3017,2389,1.268440,1.301174,3.493173,26,0.832161,0.898456,0.937699,0.744910,0.800836,0.932577,0.387549,0.970523,0.379264,399.562491
1767996,2070,1.8,0.75,351,228,649,351,117,370,0,2020,310,186,782,302,148,593,2122,2922,2322,1.269345,1.305246,1.797272,12,0.824916,0.915085,0.909522,0.842386,0.891827,0.907820,0.442282,0.984867,0.379264,399.562491
1767997,2070,1.8,0.75,351,228,649,351,117,370,0,2021,296,147,737,280,143,574,2035,2799,2181,1.236787,1.231282,1.727339,11,0.745643,0.921879,0.915641,0.790904,0.891715,0.918694,0.388810,0.989920,0.379264,399.562491
1767998,2070,1.8,0.75,351,228,649,351,117,370,0,2022,272,205,666,254,152,490,1876,2487,2041,1.287070,1.357176,9.949258,63,0.843552,0.951629,0.853462,0.766633,0.789073,0.917727,0.397889,0.982227,0.379264,399.562491




Municipality =  Laerdal


,init_pop,init_hps,sh_threshold,i_fcalves,i_yhinds,i_ahinds,i_mcalves,i_ystags,i_astags,rep_num,obs_year,f_calves,y_hinds,a_hinds,m_calves,y_stags,a_stags,spring_pop,tot_pop_bh,tot_pop_ah,hps_bh,hps_ah,mig%,num_migs,ws_fc,ws_yh,ws_ah,ws_mc,ws_ys,ws_as,c_yh,c_ah,scaling,L2_norm
0,1930,1.3,0.65,270,125,659,270,169,435,0,2007,257,194,610,252,163,384,1712,2203,1863,1.398416,1.469232,21.187391,122,0.841594,0.996884,0.871072,0.770452,0.914470,0.971475,0.396945,0.980493,0.324622,463.704216
1,1930,1.3,0.65,270,125,659,270,169,435,0,2008,255,166,631,262,169,405,1647,2195,1891,1.347076,1.387821,9.840287,51,0.764223,0.903181,0.890439,0.841627,0.934742,0.951397,0.377802,0.977590,0.324622,463.704216
2,1930,1.3,0.65,270,125,659,270,169,435,0,2009,258,167,631,260,168,417,1666,2242,1903,1.362745,1.363682,9.838375,51,0.822126,0.889031,0.908529,0.818123,0.883372,0.913932,0.453398,0.975509,0.324622,463.704216
3,1930,1.3,0.65,270,125,659,270,169,435,0,2010,255,158,614,246,153,395,1677,2207,1824,1.397100,1.409618,16.287146,90,0.815703,0.904223,0.890921,0.759617,0.923777,0.955399,0.500606,0.969503,0.324622,463.704216
4,1930,1.3,0.65,270,125,659,270,169,435,0,2011,263,137,645,252,128,403,1590,2186,1831,1.406391,1.473246,5.850478,29,0.731639,0.938996,0.936915,0.718798,0.860051,0.932528,0.459466,0.966220,0.324622,463.704216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767995,1630,1.6,0.55,244,91,610,244,184,254,0,2019,300,178,754,289,193,433,1947,2620,2149,1.504709,1.485660,15.378038,87,0.786892,0.879719,0.878301,0.830890,0.843656,0.981092,0.480939,0.970494,0.347771,512.583766
1767996,1630,1.6,0.55,244,91,610,244,184,254,0,2020,300,188,769,287,147,431,1930,2572,2125,1.629127,1.655392,19.899658,119,0.820781,0.940862,0.938770,0.716378,0.864183,1.000000,0.394389,0.981505,0.347771,512.583766
1767997,1630,1.6,0.55,244,91,610,244,184,254,0,2021,305,141,785,276,155,443,1897,2640,2108,1.521752,1.546781,7.176245,38,0.742215,0.859368,0.957438,0.844644,0.815378,0.955917,0.381125,0.976261,0.347771,512.583766
1767998,1630,1.6,0.55,244,91,610,244,184,254,0,2022,309,188,788,298,140,419,1894,2565,2144,1.610034,1.741556,14.631525,80,0.851703,0.949146,0.921408,0.804997,0.920455,0.923119,0.381113,0.968189,0.347771,512.583766




CPU times: user 4min 42s, sys: 8min 41s, total: 13min 23s
Wall time: 2h 1min 58s


In [22]:
# stop cluster syncronously
cluster.stop_cluster_sync()

Stopping controller
Controller stopped: {'exit_code': 0, 'pid': 6655, 'identifier': 'ipcontroller-1739956107-4x1p-6651'}
Stopping engine(s): 1739956108
engine set stopped 1739956108: {'engines': {'0': {'exit_code': 0, 'pid': 6668, 'identifier': '0'}, '1': {'exit_code': 0, 'pid': 6674, 'identifier': '1'}, '2': {'exit_code': 0, 'pid': 6675, 'identifier': '2'}, '3': {'exit_code': 0, 'pid': 6677, 'identifier': '3'}, '4': {'exit_code': 0, 'pid': 6679, 'identifier': '4'}, '5': {'exit_code': 0, 'pid': 6682, 'identifier': '5'}, '6': {'exit_code': 0, 'pid': 6685, 'identifier': '6'}, '7': {'exit_code': 0, 'pid': 6689, 'identifier': '7'}, '8': {'exit_code': 0, 'pid': 6693, 'identifier': '8'}, '9': {'exit_code': 0, 'pid': 6697, 'identifier': '9'}, '10': {'exit_code': 0, 'pid': 6703, 'identifier': '10'}, '11': {'exit_code': 0, 'pid': 6709, 'identifier': '11'}, '12': {'exit_code': 0, 'pid': 6715, 'identifier': '12'}, '13': {'exit_code': 0, 'pid': 6721, 'identifier': '13'}, '14': {'exit_code': 0, 'pi

In [23]:
cluster._is_running()

False

# Testing the sanity of the results

In [26]:
# Revoking top hits frames
save_dir = "synthetic_data/"
top_hits_Tingvoll_spring_count = pd.read_pickle(
    save_dir + "top_hits_Tingvoll_spring_count_stoch_par.pkl"
)
top_hits_Surnadal_spring_count = pd.read_pickle(
    save_dir + "top_hits_Surnadal_spring_count_stoch_par.pkl"
)
top_hits_Sunndal_spring_count = pd.read_pickle(
    save_dir + "top_hits_Sunndal_spring_count_stoch_par.pkl"
)
top_hits_Vestnes_spring_count = pd.read_pickle(
    save_dir + "top_hits_Vestnes_spring_count_stoch_par.pkl"
)
top_hits_Laerdal_spring_count = pd.read_pickle(
    save_dir + "top_hits_Laerdal_spring_count_stoch_par.pkl"
)

top_hits_spring_count = [
    top_hits_Tingvoll_spring_count,
    top_hits_Surnadal_spring_count,
    top_hits_Sunndal_spring_count,
    top_hits_Vestnes_spring_count,
    top_hits_Laerdal_spring_count,
]

In [28]:
save_dir = "synthetic_data/"
municipalities = ["Tingvoll", "Surnadal", "Sunndal", "Vestnes", "Laerdal"]
for q in range(len(top_hits_spring_count)):
    # Read through and test for consistent periodicity regarding years
    # which checks that there are no conflicts associated with the sorting of the data frames
    counter = 0
    for i in range(0, len(top_hits_spring_count[q]), number_of_years):
        for j in range(number_of_years):
            if top_hits_spring_count[q]["obs_year"][i + j] != (2007 + j):
                # print('i:', i, 'j:', j, 'obs_year value:', top_hits_spring_count[q]['obs_year'][i+j])
                counter = counter + 1
    print("Number of troubles in " + municipalities[q] + ":", counter)

Number of troubles in Tingvoll: 0
Number of troubles in Surnadal: 0
Number of troubles in Sunndal: 0
Number of troubles in Vestnes: 0
Number of troubles in Laerdal: 0
